In [ ]:
#Libraries imports
import torch
import edward
from edward.models import Categorical
import pandas as pd
#from sklearn.metrics import mean_squared_error
import time
import numpy as np
import random
#import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf

In [ ]:
#Load test data and necessary data for prediction inspection

with open('/tmp/PredictionPDF.pt','rb') as f:
         probs = torch.load(f)
#with open('/tmp/PDFCenters.pt','rb') as f:
#         centers = torch.load(f)
with open('../data/Wall/test_data_2sensors_1hot.pt','rb') as f:
      test_ = torch.load(f)



test_data = test_ 
N = len(test_data)

In [ ]:
#Inspect posterior predictive distribution

y_post = []
sigma_y = 1.0
n_samples = 3000


t0 = time.time()
for i in xrange(N):
    #print("Forming the posterior predictive distribution for test data point", i+1, "/", len(test_data), "...")
    y_post.append(Categorical(probs = probs[i]))
    
print("Took altogether", np.int(time.time() - t0), "secs.")

t0 = time.time()                                                                                                                                                                                                                                                                                                                                                                                                                                                            
print("Sampling the posterior predictive distribution for", len(test_data), "test data points...")
#posteriorsamplenodes = tf.stack([y_post[i].sample(n_samples) for i in range(N)], axis=1)
#Posterior samples contain n_samples samples from each posterior predictive distribution, shape = n_samples x N
psnodes = np.zeros((n_samples,N))
with tf.Session().as_default():
    for i in xrange(N):
        psnodes[:,i] = y_post[i].sample(n_samples).eval()
        
    posteriorsamples = pd.DataFrame(psnodes)
    
#print(posteriorsamples)
print("Took", np.int(time.time() - t0), "secs.")

In [ ]:
#Inspect prediction statistics

#Predictions are the mean over the #n_samples predicted values for each test data point
predictions = posteriorsamples.mean()
predictions_low = posteriorsamples.quantile(0.01)
predictions_high = posteriorsamples.quantile(0.99)

results = pd.DataFrame()
results['Bayesian_prediction'] = predictions    
results['Bayesian_prediction_low'] = predictions_low
results['Bayesian_prediction_high'] = predictions_high

#Results contain the accuracy of the predictions on the test data points
print("Accuracy on test data for Bayesian neural network:")
print(len(np.where(results['Bayesian_prediction']==np.argmax(test_data[:,2:6],axis=1))[0])/float(len(test_data[:,2]))*100)
#results

In [ ]:
#Visualise one (random) prediction and associated uncertainty

test_sample_number = 1 #np.random.choice(range(len(test_data)))
plt.figure(figsize=(15,5))

_, _, histogram = plt.hist(posteriorsamples[test_sample_number].values, bins=[0,1,2,3], density=True, align='left',rwidth=0.3)

plt.xlabel("Predicted class")
plt.ylabel("PDF (counts/n_samples x binwidth)")
plt.axvline(results.loc[test_sample_number, 'Bayesian_prediction'], color='g', linewidth=4, label="Bayesian NN")
plt.axvline(test_data[test_sample_number,2], color='k', linewidth=4, label="Ground truth")
plt.legend()
plt.title("Results for test data point " + str(test_sample_number))